In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
import keras 
from keras.layers import Conv2D
from keras.models import Sequential
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import cv2
import re
import random
random.seed(0)
np.random.seed(0)

In [ ]:
def resized_image(data):
    flat = []
    for i in data:
        # Open and resize the image using PIL
        image = Image.open(i)
        width, height = image.size
        new_width = 224
        new_height = 224
        resized_image = image.resize((new_width, new_height))
        
        # Convert to a NumPy array and normalize
        img_array = np.array(resized_image) / 255.0
        
        # Append the processed image to the list
        flat.append(img_array)
    
    # Reshape the list of image arrays into a 4D array
    processed_images = np.array(flat).reshape(-1, 224, 224, 3)
    
    return processed_images

In [ ]:
def deep_pipeline(data):
    flat = []
    for i in data:
        img = plt.imread(i)
        img = img/255.
        flat.append(img)
    flat =  np.array(flat)    
    flat = flat.reshape(-1,224,224,3)       
    return flat

In [ ]:
for root, _, files in os.walk("/kaggle/input/rice-leaf-diseases"):
    if len(root) > len("/kaggle/input/rice-leaf-diseases/rice_leaf_diseases"):
        label = root[len("rice-leaf-diseases")+1:]
        print(label)

In [ ]:
import h5py
from PIL import Image, ImageOps, ImageEnhance
import os

def adjust_contrast(img, contrast_factor):
    enhancer = ImageEnhance.Contrast(img)
    contrast_adjusted_img = enhancer.enhance(contrast_factor)
    return Image.blend(img, contrast_adjusted_img, contrast_factor)

# Function to read and convert a PNG file to a dataset
def convert_jpg_to_array(image_path):
    image = Image.open(image_path)
    # image = adjust_contrast(image, 1.5)
    image_data = np.array(image)
    return image_data
    
def convert_images_to_dataset():
    ls = []
    image_data=[]
    for root, _, files in os.walk("/kaggle/input/rice-leaf-diseases"):
        if len(root) > len("/kaggle/input/rice-leaf-diseases/rice_leaf_diseases"):
            label = root[len("/kaggle/input/rice-leaf-diseases/rice_leaf_diseases")+1:]
        for filename in files:
            image_path = os.path.join(root, filename)
            image_data.append(image_path)
            if label == "Bacterial leaf blight":
                ls.append(0)
            elif label == "Brown spot":
                ls.append(1)
            elif label == "Leaf smut":
                ls.append(2)
    return image_data, ls


In [ ]:
image_data, ls = convert_images_to_dataset()
df = pd.DataFrame()
df["image_path"] = image_data
df["label"] = ls
df = df.sample(frac=1, random_state=42)  # Set a random_state for reproducibility
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
ohencoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
ohlabel = pd.DataFrame(ohencoder.fit_transform(df[['label']]),dtype = 'float64',columns = ['label0','label1','label2'])
label_X = df.copy()
df = pd.concat([df,ohlabel],axis = 1)
new_df = df.drop(['label'],axis = 1)

In [ ]:
train,test = train_test_split(new_df,test_size=0.2,random_state=32,shuffle = True)

In [ ]:
X_train = train['image_path'].values
y_train = train.drop(['image_path'],axis=1).values
X_test = test['image_path'].values
y_test = test.drop(['image_path'],axis=1).values

In [ ]:
dx_train = resized_image(X_train)
dx_test = resized_image(X_test)

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50V2(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=3,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(3, activation='softmax'))

In [ ]:
resnet_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.summary()

In [ ]:
epochs=10
history = resnet_model.fit(dx_train,y_train,epochs=50,validation_split=0.3,batch_size=32)

In [ ]:
score = resnet_model.evaluate(dx_test,y_test)
print("accuracy: ", score[1])

In [ ]:
def deepmodelpipeline(imagepath,model = resnet_model,label=[-1]):
    pdict = {0:"Bacterial leaf blight",1:"Brown spot",2:"Leaf smut"}
    pred_x = resized_image([imagepath])
    prediction = model.predict(pred_x)
    pred = np.argmax(prediction[0])
    plt.imshow(plt.imread(imagepath))
    if (label[0]!=-1):
        plt.title("prediction : {0} % {1:.2f} \ntrue        : {2}".format(pdict[pred],prediction[0,pred]*100,pdict[np.argmax(label)]))
    else:
        plt.title("prediction : {0}, % {1:.2f}".format(pdict[pred],prediction[0,pred]*100))

In [ ]:
image_path = "/kaggle/input/rice-leaf-diseases/rice_leaf_diseases/Brown spot/DSC_0100.jpg"
deepmodelpipeline(image_path)

In [ ]:
import pickle
with open("model.pkl", 'wb') as model_file:
    pickle.dump(resnet_model, model_file)
